In [7]:
# client.py
from dotenv import load_dotenv
import os
from openai import OpenAI, AzureOpenAI

load_dotenv()

# --- Direct OpenAI client (optional) ---
#openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- Azure OpenAI client ---
azure_client = AzureOpenAI(
    api_version="2025-03-01-preview",
    azure_endpoint="https://ai-ethan0241ai492715927424.openai.azure.com/",
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

client = azure_client

# --- Default client (choose one) ---
# You can toggle this dynamically:
# USE_AZURE = True
# client = azure_client if USE_AZURE else openai_client


In [ ]:
import pandas as pd  
from pydantic import BaseModel
from concurrent.futures import ThreadPoolExecutor
# from client import client

# Define schema for structured output
class BinaryOutcome(BaseModel):
    call_id: str
    binary_explanation: str
    binary_label: str

messages = [
        {
            "role": "system",
            "content": f"""
            You are a precise binary classifier. Answer strictly 'yes' or 'no' 
            based only on evidence in the transcript. Do not make assumptions. 
            Always respond with valid JSON.
            """
        },
        {
            "role": "user",
            "content": f"""
            Question: Your task is to determine whether a given call transcript represents an **Inbound** or **Outbound** call. **Inbound:** The customer or client initiates the call to the company (e.g., seeking support, billing, or service help). **Outbound:** The agent or company initiates the call to the customer (e.g., sales outreach, renewal attempt, retention, or follow-up). 
            Transcript: this is a sample transcript haha, lets pretend I am an inbound call
            
            Respond ONLY as JSON:
            {{
                "binary_label": "yes" or "no",
                "binary_explanation": "Reasoning citing evidence from the transcript"
            }}
            """,
        },
    ]

response = client.responses.parse(
            model="gpt-5-mini",
            input=messages,
            text_format=BinaryOutcome,
            temperature=1.0
        )



In [12]:
print(response)

ParsedResponse[BinaryOutcome](id='resp_0e3a795abc9d13150069024cfdbb188197ad240b8aa39511ae', created_at=1761758461.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-mini', object='response', output=[ResponseReasoningItem(id='rs_0e3a795abc9d13150069024cfe520c8197a9914c281990352d', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ParsedResponseOutputMessage[BinaryOutcome](id='msg_0e3a795abc9d13150069024d09f7cc8197ab0b60636d2540a7', content=[ParsedResponseOutputText[BinaryOutcome](annotations=[], text='{"call_id":"sample_001","binary_explanation":"The transcript explicitly states \'lets pretend I am an inbound call\', indicating the speaker is portraying an inbound call.","binary_label":"yes"}', type='output_text', logprobs=[], parsed=BinaryOutcome(call_id='sample_001', binary_explanation="The transcript explicitly states 'lets pretend I am an inbound call', indicating the speaker is portraying an inbound call.", binary_la

In [13]:
print(response.output_parsed)

call_id='sample_001' binary_explanation="The transcript explicitly states 'lets pretend I am an inbound call', indicating the speaker is portraying an inbound call." binary_label='yes'
